In [46]:
import os
import pandas as pd
import csv
import functools as ft
import praw
import _csv

In [17]:
# constants
hpc_root = "/work/akilby/drug_pricing_project/opiates/opiates/"
local_root = "/Users/cccdenhart/Documents/work/kilby_ra/drug_pricing/sample_data"
thread_fn1 = os.path.join(local_root, "threads/r_opiates_1552621734_1553226534_t1553226534.csv")
thread_fn2 = os.path.join(local_root, "threads/r_opiates_1447632000_1448928000.csv")
comment_fn1 = os.path.join(local_root, "comments/opiates_5ps0hy.csv")
comment_fn2 = os.path.join(local_root, "comments/opiates_yesxg.csv")

In [6]:
# read threads and comments into dataframes
thread_df = read_all_files(os.path.join(root, "threads"))
comm_df = read_all_files(os.path.join(root, "comments", "complete"))

NameError: name 'read_all_files' is not defined

In [18]:
thread_df1 = pd.read_csv(thread_fn1, header=None)
thread_df2 = pd.read_csv(thread_fn2, header=None)
comm_df1 = pd.read_csv(comment_fn1, header=None)
comm_df2 = pd.read_csv(comment_fn2, header=None)

In [63]:
r = csv.reader(open(thread_fn1))

In [64]:
next(r)

['b408hw',
 'https://i.redd.it/oxa29eljdln21.jpg',
 '2',
 'https://redd.it/b408hw',
 'fentfiends',
 '12 gauge is the best out omg be careful if you run into it!',
 '',
 '1553226243.0']

In [19]:
thread_df1.head()

,0,1,2,3,4,5,6,7
0,b408hw,https://i.redd.it/oxa29eljdln21.jpg,2,https://redd.it/b408hw,fentfiends,12 gauge is the best out omg be careful if you...,NaN,1.553226e+09
1,b4044w,https://i.redd.it/ldj2l8c6bln21.jpg,0,https://redd.it/b4044w,ty_mudlife710,Couple gabapentin 300mg and an 8mg subutex.. D...,NaN,1.553225e+09
2,b4036o,https://www.reddit.com/r/opiates/comments/b403...,11,https://redd.it/b4036o,thecrashedschoolbus,Will my meds be good wit opiates ?,"I take 10mg buspar and 20mg celexa , I was pla...",1.553225e+09
3,b402ib,https://i.redd.it/gddus4cbaln21.jpg,0,https://redd.it/b402ib,Sake-and-Opium,You guys might appreciate this. Bc this sub wa...,NaN,1.553225e+09
4,b400k3,https://i.redd.it/hdso6si99ln21.jpg,2,https://redd.it/b400k3,puppypink0,Found an old prescription from last year. CVS ...,NaN,1.553225e+09


In [20]:
thread_df2.head()

,0,1,2,3,4,5,6,7
0,3t16ng,http://i.imgur.com/b2UuEhL.jpg,16,https://redd.it/3t16ng,User_No_Name,Dope porn. Happy Monday! Time to scuk today's ...,NaN,1.447690e+09
1,3t13hq,https://www.reddit.com/r/opiates/comments/3t13...,3,https://redd.it/3t13hq,OxyJay,It's a sad reality,But I feel like the only reason i come here an...,1.447688e+09
2,3t12q9,https://www.reddit.com/r/opiates/comments/3t12...,15,https://redd.it/3t12q9,XBropiate91,Mincing opana er to a powder?,I'm curious as to how to do this. I don't own ...,1.447688e+09
3,3t11si,https://www.youtube.com/watch?v=OkkuAyhJizs,11,https://redd.it/3t11si,hamonz123,"[Music] Shootin up, Breaking down - Chad Hates...",NaN,1.447687e+09
4,3t0z5l,https://www.reddit.com/r/opiates/comments/3t0z...,18,https://redd.it/3t0z5l,MyNameIsntSteven,Killing 6 hours,God damn this is gonna be tough. There's nothi...,1.447686e+09


In [29]:
praw.Reddit.submission(id="3t16ng")

TypeError: submission() missing 1 required positional argument: 'self'

In [21]:
comm_df1.head()

,0,1,2,3,4,5
0,dctcb6r,https://www.reddit.com/r/opiates/comments/5ps0...,t3_5ps0hy,"Fuck autocorrect. It was meant to read "" if sa...",Dxmyellowbelly,1.485209e+09
1,dctsr6h,https://www.reddit.com/r/opiates/comments/5ps0...,t3_5ps0hy,If Carl Sagan gave out baggies\n,spadeindigo47,1.485229e+09
2,dctx0vp,https://www.reddit.com/r/opiates/comments/5ps0...,t1_dctcb6r,Satan autocorrected to Sagan that'd weird. Car...,Quality_Con_Troll,1.485235e+09
3,dctzulq,https://www.reddit.com/r/opiates/comments/5ps0...,t1_dctsr6h,Classic,Dxmyellowbelly,1.485241e+09


In [22]:
comm_df2.head()

,0,1,2,3,4,5
0,c5uwsok,https://www.reddit.com/r/opiates/comments/yesx...,t3_yesxg,[deleted],NaN,1.345262e+09
1,c5uwow1,https://www.reddit.com/r/opiates/comments/yesx...,t3_yesxg,The only thing that would really fuck them up ...,gratefulgrux,1.345262e+09
2,c5uw0s7,https://www.reddit.com/r/opiates/comments/yesx...,t3_yesxg,they won't harm you in any way,scott_halls_beer,1.345258e+09
3,c5uwujp,https://www.reddit.com/r/opiates/comments/yesx...,t3_yesxg,"Not medical advice, but as far as I know, if s...",allesklaar,1.345262e+09
4,c5uxtc9,https://www.reddit.com/r/opiates/comments/yesx...,t3_yesxg,Took some codeine my buddy found in his medici...,thatpost,1.345267e+09
